In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from sklearn.metrics.pairwise import linear_kernel
import warnings; warnings.simplefilter('ignore')

In C:\Users\PKN\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\PKN\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\PKN\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\PKN\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\PKN\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
Th

In [ ]:
combined_df = pd.read_csv('F:/Visual Analytics and Applications/Project/Recommender_amit/movielens-imdb-combined/movielens-imdb-combined.csv')

In [4]:
combined_df.head()

,movielens_movieId,movielens_title,movielens_genres,imdb_link,movielens_userId,movielens_rating,movielens_timestamp_rating,movielens_user_review,movielens_timestamp_tag,imdb_director_name,...,imdb_actor_3_name,imdb_plot_keywords,imdb_num_user_for_reviews,imdb_language,imdb_country,imdb_budget,imdb_title_year,imdb_actor_2_facebook_likes,imdb_score,imdb_movie_facebook_likes
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,https://www.imdb.com/title/tt0114709,1040,2.5,1514919511,animated,1514919574,John Lasseter,...,Jim Varney,claw crane|cowboy|jealousy|rivalry|toy,391.0,English,USA,30000000.0,1995.0,1000.0,8.3,0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,https://www.imdb.com/title/tt0114709,1040,2.5,1514919511,buddy movie,1514919577,John Lasseter,...,Jim Varney,claw crane|cowboy|jealousy|rivalry|toy,391.0,English,USA,30000000.0,1995.0,1000.0,8.3,0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,https://www.imdb.com/title/tt0114709,1040,2.5,1514919511,Cartoon,1514919589,John Lasseter,...,Jim Varney,claw crane|cowboy|jealousy|rivalry|toy,391.0,English,USA,30000000.0,1995.0,1000.0,8.3,0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,https://www.imdb.com/title/tt0114709,1040,2.5,1514919511,cgi,1514919556,John Lasseter,...,Jim Varney,claw crane|cowboy|jealousy|rivalry|toy,391.0,English,USA,30000000.0,1995.0,1000.0,8.3,0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,https://www.imdb.com/title/tt0114709,1040,2.5,1514919511,comedy,1514919569,John Lasseter,...,Jim Varney,claw crane|cowboy|jealousy|rivalry|toy,391.0,English,USA,30000000.0,1995.0,1000.0,8.3,0


In [5]:
vote_counts = combined_df[combined_df['imdb_num_critic_for_reviews'].notnull()]['imdb_num_critic_for_reviews'].astype('int')
vote_averages = combined_df[combined_df['imdb_score'].notnull()]['imdb_score'].astype('int')
C = vote_averages.mean()
C

7.0547329232647815

In [6]:
m = vote_counts.quantile(0.01)
m

42.0

In [7]:
combined_df['year'] = combined_df['movielens_title'].apply(lambda x: str(x).replace(")","").split(" (")[1])

In [8]:
combined_df['movie'] = combined_df['movielens_title'].apply(lambda x: str(x).replace(")","").split(" (")[0])

In [9]:
combined_df.columns

Index(['movielens_movieId', 'movielens_title', 'movielens_genres', 'imdb_link',
       'movielens_userId', 'movielens_rating', 'movielens_timestamp_rating',
       'movielens_user_review', 'movielens_timestamp_tag',
       'imdb_director_name', 'imdb_num_critic_for_reviews', 'imdb_duration',
       'imdb_director_facebook_likes', 'imdb_actor_3_facebook_likes',
       'imdb_actor_2_name', 'imdb_actor_1_facebook_likes', 'imdb_gross',
       'imdb_genres', 'imdb_actor_1_name', 'imdb_movie_title',
       'imdb_num_voted_users', 'imdb_cast_total_facebook_likes',
       'imdb_actor_3_name', 'imdb_plot_keywords', 'imdb_num_user_for_reviews',
       'imdb_language', 'imdb_country', 'imdb_budget', 'imdb_title_year',
       'imdb_actor_2_facebook_likes', 'imdb_score',
       'imdb_movie_facebook_likes', 'year', 'movie'],
      dtype='object')

In [10]:
qualified = combined_df[(combined_df['imdb_num_critic_for_reviews'] >= m) & (combined_df['imdb_num_critic_for_reviews'].notnull()) & (combined_df['imdb_score'].notnull())][['movie', 'year','imdb_plot_keywords','imdb_director_name', 'imdb_num_critic_for_reviews', 'imdb_score','imdb_actor_1_facebook_likes', 'imdb_director_facebook_likes','imdb_actor_1_name','imdb_actor_2_name','imdb_movie_facebook_likes', 'movielens_genres']]
qualified['imdb_num_critic_for_reviews'] = qualified['imdb_num_critic_for_reviews'].astype('int')
qualified['imdb_score'] = qualified['imdb_score'].astype('int')
qualified.shape

(564594, 12)

In [11]:
stemmer = SnowballStemmer('english')

In [12]:
def weighted_rating(x):
    v = x['imdb_num_critic_for_reviews']
    R = x['imdb_score']
    return (v/(v+m) * R) + (m/(m+v) * C)
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [13]:
qualified = qualified.sort_values('wr', ascending=False)

In [14]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [15]:
features = ['imdb_actor_1_name','imdb_actor_2_name', 'imdb_plot_keywords', 'imdb_director_name', 'movielens_genres']

for feature in features:
    qualified[feature] = qualified[feature].apply(clean_data)

In [16]:
qualified['imdb_plot_keywords'] = qualified['imdb_plot_keywords'].str.replace('|',' ')

In [17]:
qualified['movielens_genres'] = qualified['movielens_genres'].str.replace('|',' ')

In [110]:
def create_soup(x):
    return ' '.join(x['imdb_plot_keywords']) + ' ' + ' '.join(x['imdb_actor_1_name']) + ' ' + ' '.join(x['imdb_actor_2_name']) + ' ' + x['imdb_director_name'] + ' ' + ' '.join(x['movielens_genres'])

In [18]:
def create_soup1(x):
    return ''.join(x['imdb_plot_keywords']) + ' ' + ''.join(x['imdb_actor_1_name']) + ' ' + ''.join(x['imdb_actor_2_name']) + ' ' + x['imdb_director_name'] + ' ' + ''.join(x['movielens_genres'])

In [19]:
qualified['imdb_plot_keywords']

365721    basedoncomicbook dccomics psychopath stardiedb...
365934    basedoncomicbook dccomics psychopath stardiedb...
365922    basedoncomicbook dccomics psychopath stardiedb...
365923    basedoncomicbook dccomics psychopath stardiedb...
365924    basedoncomicbook dccomics psychopath stardiedb...
                                ...                        
377832    disaster glass mousetrap naturaldisaster scato...
377831    disaster glass mousetrap naturaldisaster scato...
377830    disaster glass mousetrap naturaldisaster scato...
377829    disaster glass mousetrap naturaldisaster scato...
377833    disaster glass mousetrap naturaldisaster scato...
Name: imdb_plot_keywords, Length: 564594, dtype: object

In [20]:
qualified['movielens_genres']

365721    action crime drama imax
365934    action crime drama imax
365922    action crime drama imax
365923    action crime drama imax
365924    action crime drama imax
                   ...           
377832                     comedy
377831                     comedy
377830                     comedy
377829                     comedy
377833                     comedy
Name: movielens_genres, Length: 564594, dtype: object

In [21]:
qualified['soup'] = qualified.apply(create_soup1, axis=1)

In [22]:
pd.set_option('display.max_colwidth', -1)

In [23]:
qualified[['soup']].head(2)

,soup
365721,basedoncomicbook dccomics psychopath stardiedbeforerelease urbansetting christianbale heathledger christophernolan action crime drama imax
365934,basedoncomicbook dccomics psychopath stardiedbeforerelease urbansetting christianbale heathledger christophernolan action crime drama imax


In [24]:
qualified = qualified.drop_duplicates('movie',keep = 'first')

In [25]:
count = CountVectorizer(stop_words = 'english')
count_matrix = count.fit_transform(qualified['soup'])

In [26]:
count_matrix.shape

(3686, 11553)

In [27]:
count_matrix

<3686x11553 sparse matrix of type '<class 'numpy.int64'>'
	with 39351 stored elements in Compressed Sparse Row format>

In [28]:
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [29]:
qualified = qualified.reset_index()
indices = pd.Series(qualified.index, index=qualified['movie'])

In [30]:
indices = pd.Series(qualified.index, index=qualified['movie']).drop_duplicates()

In [31]:
print(indices[:50])

movie
Dark Knight, The              0 
Godfather, The                1 
Shawshank Redemption, The     2 
Godfather: Part II, The       3 
Decalogue, The                4 
Dark Knight Rises, The        5 
Django Unchained              6 
Mad Max: Fury Road            7 
Interstellar                  8 
Avengers, The                 9 
Black Swan                    10
Guardians of the Galaxy       11
Inception                     12
Wolf of Wall Street, The      13
12 Years a Slave              14
Deadpool                      15
The Artist                    16
Her                           17
The Martian                   18
Gone Girl                     19
The Revenant                  20
Life of Pi                    21
Boyhood                       22
X-Men: Days of Future Past    23
Grand Budapest Hotel, The     24
Inside Out                    25
Whiplash                      26
V for Vendetta                27
Star Trek                     28
Captain America: Civil War    29
Shut

In [32]:
qualified['movie'].unique()

array(['Dark Knight, The', 'Godfather, The', 'Shawshank Redemption, The',
       ..., 'Justin Bieber: Never Say Never', 'Battlefield Earth',
       'Disaster Movie'], dtype=object)

Function to get top 5 similar movies based on cosine similarity

In [34]:
def get_recommendations(title, cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 5 most similar movies
    sim_scores = sim_scores[1:6]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 5 most similar movies
    return qualified['movie'].iloc[movie_indices]

In [37]:
get_recommendations('Interstellar', cosine_sim=cosine_sim2)

12      Inception      
815     Contact        
1580    Cargo          
2126    Dune           
3001    Black Hole, The
Name: movie, dtype: object

In [134]:
movies, scores = get_recommendations('Terminator 2: Judgment Day', cosine_sim=cosine_sim2)

In [135]:
movies

2891    Timecop                           
1394    Looper                            
1409    Avatar                            
927     Starship Troopers                 
1653    Escape from the Planet of the Apes
Name: movie, dtype: object

In [136]:
scores

[(2891, 0.4351941398892446),
 (1394, 0.41812100500354543),
 (1409, 0.41812100500354543),
 (927, 0.36363636363636365),
 (1653, 0.36363636363636365)]

In [137]:
movies = pd.DataFrame(movies)

In [138]:
movie_trans = movies.transpose()

In [139]:
movie_trans.columns = ['Movie1','Movie2','Movie3','Movie4','Movie5']

In [140]:
movie_trans

,Movie1,Movie2,Movie3,Movie4,Movie5
movie,Timecop,Looper,Avatar,Starship Troopers,Escape from the Planet of the Apes


In [141]:
scores

[(2891, 0.4351941398892446),
 (1394, 0.41812100500354543),
 (1409, 0.41812100500354543),
 (927, 0.36363636363636365),
 (1653, 0.36363636363636365)]

In [142]:
scores =  list(zip(*scores)) 

In [143]:
scores

[(2891, 1394, 1409, 927, 1653),
 (0.4351941398892446,
  0.41812100500354543,
  0.41812100500354543,
  0.36363636363636365,
  0.36363636363636365)]

In [144]:
match = pd.DataFrame(scores[1])

In [145]:
match

,0
0,0.435194
1,0.418121
2,0.418121
3,0.363636
4,0.363636


In [146]:
match_fs = match.copy()
match_fs.columns = ['score']
match_fs['percent_score']  = match_fs['score']*100
del match_fs['score']

In [147]:
match_fs

,percent_score
0,43.519414
1,41.812101
2,41.812101
3,36.363636
4,36.363636


In [148]:
match_fs = match_fs.transpose()

In [149]:
match_fs.columns = ['Movie1_FS','Movie2_FS','Movie3_FS','Movie4_FS','Movie5_FS']

In [150]:
match_fs 

,Movie1_FS,Movie2_FS,Movie3_FS,Movie4_FS,Movie5_FS
percent_score,43.519414,41.812101,41.812101,36.363636,36.363636


In [151]:
movie_trans = movie_trans.reset_index()
match_fs = match_fs.reset_index()

In [152]:
recommenderoutput = pd.concat([movie_trans[['Movie1','Movie2','Movie3','Movie4','Movie5']],
                               match_fs[['Movie1_FS','Movie2_FS','Movie3_FS','Movie4_FS','Movie5_FS']]],axis = 1)

#### 5 most similar movies

In [153]:
recommenderoutput

,Movie1,Movie2,Movie3,Movie4,Movie5,Movie1_FS,Movie2_FS,Movie3_FS,Movie4_FS,Movie5_FS
0,Timecop,Looper,Avatar,Starship Troopers,Escape from the Planet of the Apes,43.519414,41.812101,41.812101,36.363636,36.363636
